In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
df_market_data.columns

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h',
                                                                    'price_change_percentage_7d',
                                                                    'price_change_percentage_14d',
                                                                    'price_change_percentage_30d',
                                                                    'price_change_percentage_60d',
                                                                    'price_change_percentage_200d',
                                                                    'price_change_percentage_1y']
])

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(market_data_scaled,
                                    columns=['price_change_percentage_24h',
                                             'price_change_percentage_7d',
                                             'price_change_percentage_14d',
                                             'price_change_percentage_30d',
                                             'price_change_percentage_60d',
                                             'price_change_percentage_200d',
                                             'price_change_percentage_1y'])

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()


In [ ]:
# Understand the diffence between original data and scaled data and their std
print(df_market_data.price_change_percentage_7d.min(), df_market_data.price_change_percentage_7d.max(), df_market_data.price_change_percentage_7d.std())
print(df_market_data.price_change_percentage_14d.min(), df_market_data.price_change_percentage_14d.max(),df_market_data.price_change_percentage_14d.std())
print(df_market_data.price_change_percentage_30d.min(), df_market_data.price_change_percentage_30d.max(),df_market_data.price_change_percentage_30d.std())

In [ ]:
# Understand the diffence between original data and scaled data and their std
print(df_market_data_scaled.price_change_percentage_7d.min(), df_market_data_scaled.price_change_percentage_7d.max(), df_market_data_scaled.price_change_percentage_7d.std())
print(df_market_data_scaled.price_change_percentage_14d.min(), df_market_data_scaled.price_change_percentage_14d.max(),df_market_data_scaled.price_change_percentage_14d.std())
print(df_market_data_scaled.price_change_percentage_30d.min(), df_market_data_scaled.price_change_percentage_30d.max(),df_market_data_scaled.price_change_percentage_30d.std())

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)



In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head(7)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_plot =df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
elbow_plot 

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** It appears that the optimal value for k, the number of clusters, is 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data

k_lower = model.predict(df_market_data_scaled)
# Print the resulting array of cluster values.
print(k_lower)

In [ ]:
# Create a copy of the DataFrame
spread_df_predictions = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
spread_df_predictions['clusters_lower']= k_lower

# Display sample data
spread_df_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_scaled_plot =spread_df_predictions.hvplot.scatter(
     x="price_change_percentage_24h",
    # x="price_change_percentage_30d",
     y="price_change_percentage_7d",
    # y = "price_change_percentage_60d",
    by="clusters_lower",
    hover_cols=['coin_id']
).opts(yformatter="%.0f")
market_data_scaled_plot 

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled)

market_data_pca[:5]

In [ ]:
# Creating a DataFrame with the PCA data
df_market_data_pca =pd.DataFrame(market_data_pca, columns=["PCA1", "PCA2" , "PCA3"])
# Set the coinid column as index 
# Copy the crypto names from the original data
df_market_data_pca["coin_id"]=df_market_data.index
df_market_data_pca=df_market_data_pca.set_index("coin_id")
# View the first five rows of the DataFrame. 
df_market_data_pca.head()

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

In [ ]:
import numpy as np
total_explained_variance = np.sum(pca.explained_variance_ratio_)
total_explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 89%

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_pca = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca: 
    k_model = KMeans(n_clusters=i,random_state=0)
    k_model.fit(df_market_data_pca)
    inertia.append(k_model.inertia_)

In [ ]:

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k_pca': k_pca , 'inertia': inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data)
df_elbow_pca.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca_plot = df_elbow_pca.hvplot.line(
    x="k_pca", 
    y="inertia", 
    title="Elbow Curve for the PCA data", 
    xticks=k
)
elbow_pca_plot

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 2


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best K value in original data was 4 and now based on elbow curve plot the best k value is 2.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=2, random_state=0)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_predict = model.predict(df_market_data_pca)
# Print the resulting array of cluster values.
k_predict

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_market_data_pca_prediction = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_data_pca_prediction["pca_prediction"]=k_predict

# Display sample data
df_market_data_pca_prediction.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_pca_plot=df_market_data_pca_prediction.hvplot.scatter(
    x="PCA1",
    # x = "PCA3",
    y="PCA2",
    #y ="PCA3",
    by="pca_prediction",
    hover_cols=['coin_id']
)
market_data_pca_plot


In [ ]:
#When we are having 3 components it's better to have 3D plot 

import plotly.express as px
df_market_data_pca_prediction = df_market_data_pca_prediction.reset_index()
fig = px.scatter_3d(df_market_data_pca_prediction, x='PCA1', y='PCA2', z='PCA3' ,
                    color='pca_prediction' , hover_data=['coin_id'])

fig.show()

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_pca_plot

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
market_data_scaled_plot + market_data_pca_plot


#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Upon reviewing the visual results of the cluster analysis, it became evident that reducing the number of features for K-Means clustering had a profound effect. Initially, the elbow curve suggested an optimal K value of 4, leading to the formation of 4 clusters. However, there were two clusters  each with only one data point, while the remaining two clusters lacked clear separation boundaries. An optimal K value of 2 yielded a more refined and accurate clustering representation.